#### Loading packages

In [1]:
import glob
import errno
import string
import numpy as np
import csv
import pandas as pd
import time

#### Creating corpus

In [2]:
truth = []
path = 'data/ground_truth/*.txt'
files = glob.glob(path)
for name in files:
    try:
        with open(name, encoding='utf8') as f:
            for line in f:
                out = line.translate(str.maketrans('', '', string.punctuation))
                out = ''.join([i for i in out if not i.isdigit()])
                out = out.lower().split()
                truth.extend(out)
    except IOError as exc:
        if exc.errno != errno.EISDIR:
            raise
np.savetxt('output/truth_corpus.dat', truth, fmt='%s', encoding='utf8')

tess = []
path = 'data/tesseract/*.txt'
files = glob.glob(path)
for name in files:
    try:
        with open(name, encoding='utf8') as f:
            for line in f:
                out = line.translate(str.maketrans('', '', string.punctuation))
                out = ''.join([i for i in out if not i.isdigit()])
                out = out.lower().split()
                tess.extend(out)
    except IOError as exc:
        if exc.errno != errno.EISDIR:
            raise
np.savetxt('output/tess_corpus.dat', tess, fmt='%s', encoding='utf8')

#### Reading data for feature extraction

In [3]:
Error = []
Truth = []
pair = []
with open('data/Error_df_rules_based.csv', encoding='utf8') as f:
    csv_reader = csv.reader(f, delimiter=',')
    for row in csv_reader:
        err = row[1].lower()
        trt = row[2].lower()
        if err != trt and [err, trt] not in pair:
            Error.append(err)
            Truth.append(trt)
            pair.append([err, trt])
            
Error = Error[2:]
Truth = Truth[2:]

#### Feature extraction

In [4]:
from lib.feature_scoring import n_gram
from lib.feature_scoring import candidate_search
from lib.feature_scoring import LED_score
from lib.feature_scoring import SS_score
from lib.feature_scoring import LP_score
from lib.feature_scoring import ECP_score
from lib.feature_scoring import RCP_score

In [7]:
W_error=['Typo']
W_truth=['Truth']
W_cand = ['Candidate']
Label = ['Label']
LED = ['led_score']
SS = ['ss_score']
LP = ['lp_score']
ECP = ['ECP_score']

n = 3 # n_gram
for i in range(len(Error)):
    w_e = Error[i]
    w_c = Truth[i]
    cand_list = candidate_search(truth, w_e)
    print('word ',i+1,', error: ', w_e, ', truth: ', w_c)
    
#    gram_list = n_gram(w_e, tess, n)
    LP_freq = []
#    ECP_freq = []
    for s in cand_list:
        lp_freq = LP_score(s, truth)
        LP_freq.append(lp_freq)
#        ecp_freq = ECP_score(gram_list, s, truth, n)
#       ECP_freq.append(ecp_freq)
        
    for j in range(len(cand_list)):
        s = cand_list[j]
        led = LED_score(w_e, s)
        ss = SS_score(w_e, s, N=3)
        lp = LP_score(s, truth)/max(LP_freq)
#        if max(ECP_freq)==0: ecp=0
#        else: ecp = ECP_score(gram_list, s, truth, n)/max(ECP_freq)
#        rcp = RCP_score(w_e, s, tess, truth)
        label = int(s == w_c)
#        print('candidate:', s, '\tscores =', '{:03.2f}'.format(led),', {:03.2f}'.format(ss),', {:06.5f}'.format(lp), '\tlabel=', label)
        W_error.append(w_e)
        W_truth.append(w_c)
        W_cand.append(s)
        Label.append(label)
        LED.append(led)
        SS.append(ss)
        LP.append(lp)
        #ECP.append(ecp)

word  1 , error:  willlam , truth:  william
word  2 , error:  nvolvng , truth:  involving
word  3 , error:  t , truth:  the
word  4 , error:  aflcid , truth:  aflcio
word  5 , error:  cmng , truth:  attaching
word  6 , error:  admlnlstratlons , truth:  administrations
word  7 , error:  d , truth:  gold
word  8 , error:  jurlsdlctlon , truth:  jurisdiction
word  9 , error:  m , truth:  to
word  10 , error:  mm , truth:  bill


KeyboardInterrupt: 

In [12]:
np.savetxt('output/feature.csv', [p for p in zip(W_error, W_truth, W_cand, LED, SS, LP, Label)], delimiter=',', fmt='%s', encoding='utf-8')

## Parameter Tunning

#### Retreive Data

In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier

In [3]:
feature_output = pd.read_csv('output/feature.csv', delimiter = ',')

In [4]:
feature_output.head(20)

,Typo,Truth,Candidate,led_score,ss_score,lp_score,Label
0,willlam,william,will,0.25,1.636364,1.000000,0
1,willlam,william,willful,0.25,1.607143,0.001712,0
2,willlam,william,william,0.75,2.142857,0.039954,1
3,willlam,william,williams,0.50,1.866667,0.003995,0
4,willlam,william,willing,0.25,1.285714,0.006849,0
5,willlam,william,wills,0.25,1.500000,0.000571,0
6,nvolvng,involving,cooling,0.25,0.857143,0.011494,0
7,nvolvng,involving,evolve,0.25,0.961538,0.045977,0
8,nvolvng,involving,evolved,0.25,0.892857,0.034483,0
9,nvolvng,involving,evolves,0.25,0.892857,0.011494,0


In [5]:
X = feature_output[feature_output.columns[3:6]]
y = feature_output["Label"]

#### Train & Test split

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,shuffle=True ,random_state=42)

In [7]:
print('X_train shape:',X_train.shape,'\n','y_train shape:',y_train.shape)

X_train shape: (1122748, 3) 
 y_train shape: (1122748,)


In [8]:
print('X_test shape:',X_test.shape,'\n','y_test shape:',y_test.shape)

X_test shape: (280687, 3) 
 y_test shape: (280687,)


## Ada Boost

In [9]:
model = AdaBoostClassifier()

## Parameters Tunning

In [22]:
parameters = {
 'n_estimators': [50, 100],
 'learning_rate' : [0.01,0.05,0.1,0.3,1],
 }

In [23]:
start = time.time()
ada_grid_search = GridSearchCV(model,parameters,cv = 3)
ada_grid_search_fit = ada_grid_search.fit(X_train, y_train)
end = time.time()
print('Time:',end - start)

Time: 1126.7511036396027


In [26]:
ada_grid_search_fit.best_params_

{'learning_rate': 0.3, 'n_estimators': 100}

In [40]:
ada_grid_search_fit.best_estimator_

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.3, n_estimators=100, random_state=None)

In [46]:
print(ada_grid_search_fit.best_score_)

0.998281003395241
